In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
import numpy as np
import string

from sklearn.linear_model import LogisticRegression
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix,classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn import metrics
from sklearn.svm import LinearSVC

In [2]:
df = pd.read_csv('redmi6.csv', encoding='latin-1')
df

,Review Title,Customer name,Rating,Date,Category,Comments,Useful
0,Another Midrange killer Smartphone by Xiaomi,Rishikumar Thakur,4.0 out of 5 stars,on 1 October 2018,Display,Another Midrange killer Smartphone by Xiaomi\n...,
1,vry small size mobile,Raza ji,3.0 out of 5 stars,on 15 September 2018,Others,All ok but vry small size mobile,7 people found this helpful
2,Full display not working in all application.,Vaibhav Patel,3.0 out of 5 stars,on 18 September 2018,Others,Quite good,7 people found this helpful
3,Value for Money,Amazon Customer,5.0 out of 5 stars,on 28 September 2018,Display,Redmi has always have been the the king of bud...,2 people found this helpful
4,Not worth for the money,Sudhakaran Wadakkancheri,2.0 out of 5 stars,on 18 September 2018,Others,worst product from MI. I am a hardcore fan of ...,6 people found this helpful
...,...,...,...,...,...,...,...
275,"Cemera quality,face unlock most important in t...",Rahul,5.0 out of 5 stars,on 19 September 2018,Others,"I like This Phone, Awesome look and design.\nI...",NaN
276,Mi is best phone,Sunil Soni,4.0 out of 5 stars,on 18 September 2018,Others,Product is avasome but invoice is note include...,NaN
277,Its a OK Phone,D.C.Padhi,3.0 out of 5 stars,on 15 September 2018,Battery,"Redmi Note4, Note5, now 6pro..It seems the old...",NaN
278,Redmi,Mahesh,5.0 out of 5 stars,on 21 September 2018,Others,I love mi,NaN


In [3]:
df.isnull().sum()

Review Title       0
Customer name      0
Rating             0
Date               0
Category           0
Comments           0
Useful           170
dtype: int64

In [4]:
df['Useful'] = df['Useful'].fillna('0 people found this helpful')
df.isnull().sum()

Review Title     0
Customer name    0
Rating           0
Date             0
Category         0
Comments         0
Useful           0
dtype: int64

In [5]:
df.duplicated().sum()

21

In [6]:
df.drop_duplicates()

,Review Title,Customer name,Rating,Date,Category,Comments,Useful
0,Another Midrange killer Smartphone by Xiaomi,Rishikumar Thakur,4.0 out of 5 stars,on 1 October 2018,Display,Another Midrange killer Smartphone by Xiaomi\n...,
1,vry small size mobile,Raza ji,3.0 out of 5 stars,on 15 September 2018,Others,All ok but vry small size mobile,7 people found this helpful
2,Full display not working in all application.,Vaibhav Patel,3.0 out of 5 stars,on 18 September 2018,Others,Quite good,7 people found this helpful
3,Value for Money,Amazon Customer,5.0 out of 5 stars,on 28 September 2018,Display,Redmi has always have been the the king of bud...,2 people found this helpful
4,Not worth for the money,Sudhakaran Wadakkancheri,2.0 out of 5 stars,on 18 September 2018,Others,worst product from MI. I am a hardcore fan of ...,6 people found this helpful
...,...,...,...,...,...,...,...
275,"Cemera quality,face unlock most important in t...",Rahul,5.0 out of 5 stars,on 19 September 2018,Others,"I like This Phone, Awesome look and design.\nI...",0 people found this helpful
276,Mi is best phone,Sunil Soni,4.0 out of 5 stars,on 18 September 2018,Others,Product is avasome but invoice is note include...,0 people found this helpful
277,Its a OK Phone,D.C.Padhi,3.0 out of 5 stars,on 15 September 2018,Battery,"Redmi Note4, Note5, now 6pro..It seems the old...",0 people found this helpful
278,Redmi,Mahesh,5.0 out of 5 stars,on 21 September 2018,Others,I love mi,0 people found this helpful


In [7]:
Rating = {'5.0 out of 5 stars':5, '4.0 out of 5 stars':4, '3.0 out of 5 stars':3, '2.0 out of 5 stars':2, '1.0 out of 5 stars':1}

df.Rating = [Rating[item] for item in df.Rating]
df.head()

,Review Title,Customer name,Rating,Date,Category,Comments,Useful
0,Another Midrange killer Smartphone by Xiaomi,Rishikumar Thakur,4,on 1 October 2018,Display,Another Midrange killer Smartphone by Xiaomi\n...,
1,vry small size mobile,Raza ji,3,on 15 September 2018,Others,All ok but vry small size mobile,7 people found this helpful
2,Full display not working in all application.,Vaibhav Patel,3,on 18 September 2018,Others,Quite good,7 people found this helpful
3,Value for Money,Amazon Customer,5,on 28 September 2018,Display,Redmi has always have been the the king of bud...,2 people found this helpful
4,Not worth for the money,Sudhakaran Wadakkancheri,2,on 18 September 2018,Others,worst product from MI. I am a hardcore fan of ...,6 people found this helpful


In [8]:
df = df[df['Rating'] != 3]
df['Sentiment'] = df['Rating'].apply(lambda rating: +1 if rating >3 else 0)
df.head()

c:\Users\Julian Andhika\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Review Title,Customer name,Rating,Date,Category,Comments,Useful,Sentiment
0,Another Midrange killer Smartphone by Xiaomi,Rishikumar Thakur,4,on 1 October 2018,Display,Another Midrange killer Smartphone by Xiaomi\n...,,1
3,Value for Money,Amazon Customer,5,on 28 September 2018,Display,Redmi has always have been the the king of bud...,2 people found this helpful,1
4,Not worth for the money,Sudhakaran Wadakkancheri,2,on 18 September 2018,Others,worst product from MI. I am a hardcore fan of ...,6 people found this helpful,0
6,Fabulous!,Irukulla bharath,4,on 17 September 2018,Display,Pros:\nnotch display\nDual camera\nFace unlock...,3 people found this helpful,1
7,Fantastic,Amazon Customer,4,on 18 September 2018,Camera,Front camera is poor rest things are good,5 people found this helpful,1


In [9]:
#Category
#Display: 1
#Camera: 2
#Battery: 3
#Delivery: 4
#Others: 5

Category = {'Display':1, 'Camera':2, 'Battery':3, 'Delivery':4, 'Others':5}
df.Category = [Category[item] for item in df.Category]
df.head()

c:\Users\Julian Andhika\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Review Title,Customer name,Rating,Date,Category,Comments,Useful,Sentiment
0,Another Midrange killer Smartphone by Xiaomi,Rishikumar Thakur,4,on 1 October 2018,1,Another Midrange killer Smartphone by Xiaomi\n...,,1
3,Value for Money,Amazon Customer,5,on 28 September 2018,1,Redmi has always have been the the king of bud...,2 people found this helpful,1
4,Not worth for the money,Sudhakaran Wadakkancheri,2,on 18 September 2018,5,worst product from MI. I am a hardcore fan of ...,6 people found this helpful,0
6,Fabulous!,Irukulla bharath,4,on 17 September 2018,1,Pros:\nnotch display\nDual camera\nFace unlock...,3 people found this helpful,1
7,Fantastic,Amazon Customer,4,on 18 September 2018,2,Front camera is poor rest things are good,5 people found this helpful,1


In [10]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

stop = stopwords.words('english')

df['tokenized_comments'] = df['Comments'].apply(lambda x: x.lower())
df['tokenized_comments'] = df['tokenized_comments'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['tokenized_comments'] = df['tokenized_comments'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in (stop)]))

df.head()

c:\Users\Julian Andhika\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\Julian Andhika\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\Users\Julian Andhika\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,Review Title,Customer name,Rating,Date,Category,Comments,Useful,Sentiment,tokenized_comments
0,Another Midrange killer Smartphone by Xiaomi,Rishikumar Thakur,4,on 1 October 2018,1,Another Midrange killer Smartphone by Xiaomi\n...,,1,another midrange killer smartphone xiaomi majo...
3,Value for Money,Amazon Customer,5,on 28 September 2018,1,Redmi has always have been the the king of bud...,2 people found this helpful,1,redmi always king budget segmentand yet anothe...
4,Not worth for the money,Sudhakaran Wadakkancheri,2,on 18 September 2018,5,worst product from MI. I am a hardcore fan of ...,6 people found this helpful,0,worst product mi hardcore fan mi one really di...
6,Fabulous!,Irukulla bharath,4,on 17 September 2018,1,Pros:\nnotch display\nDual camera\nFace unlock...,3 people found this helpful,1,pro notch display dual camera face unlock 4000...
7,Fantastic,Amazon Customer,4,on 18 September 2018,2,Front camera is poor rest things are good,5 people found this helpful,1,front camera poor rest thing good


In [11]:
# def remove_punctuation(text):
#     final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
#     return final

# df = df.dropna(subset=['tokenized_comments'])
# df['tokenized_comments'] = df['tokenized_comments'].apply(remove_punctuation)
# # df['tokenized_comments'] = df['Comments'].replace(r'\s+|\\n', ' ', regex=True) 
# df['tokenized_comments'].head()

In [12]:
# from nltk.stem import PorterStemmer

# ps = PorterStemmer()

# df['tokenized_comments'] = df['tokenized_comments'].apply(lambda x: ps.stem(x))
# df['tokenized_comments'].head()

In [13]:
dfpredictcategory = df[['tokenized_comments','Category','Sentiment']]
dfpredictcategory.head()

,tokenized_comments,Category,Sentiment
0,another midrange killer smartphone xiaomi majo...,1,1
3,redmi always king budget segmentand yet anothe...,1,1
4,worst product mi hardcore fan mi one really di...,5,0
6,pro notch display dual camera face unlock 4000...,1,1
7,front camera poor rest thing good,2,1


In [14]:
# index = df.index
# df['random_number'] = np.random.randn(len(index))

# train = dfpredictcategory[df['random_number'] <= 0.8]
# test = dfpredictcategory[df['random_number'] > 0.8]

In [15]:
# vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# train_matrix = vectorizer.fit_transform(train['tokenized_comments'])
# test_matrix = vectorizer.transform(test['tokenized_comments'])

# print(train_matrix)

In [16]:
# x_train = train_matrix
# x_test = test_matrix
# y_train = train['Category']
# y_test = test['Category']

In [17]:
vectorizer = TfidfVectorizer(token_pattern=r'\b\w+\b', ngram_range=(1, 2), max_features=5000)
XV = vectorizer.fit_transform(df['tokenized_comments'])
Y = df['Category']

print(XV)
print(Y)

  (0, 1918)	0.025212162728181484
  (0, 2444)	0.021998955732048554
  (0, 633)	0.025212162728181484
  (0, 2483)	0.025212162728181484
  (0, 1455)	0.025212162728181484
  (0, 2115)	0.025212162728181484
  (0, 1921)	0.025212162728181484
  (0, 2796)	0.025212162728181484
  (0, 2152)	0.023878561331841186
  (0, 2203)	0.025212162728181484
  (0, 1713)	0.025212162728181484
  (0, 3138)	0.025212162728181484
  (0, 2834)	0.025212162728181484
  (0, 3070)	0.025212162728181484
  (0, 196)	0.025212162728181484
  (0, 1121)	0.025212162728181484
  (0, 2843)	0.025212162728181484
  (0, 942)	0.025212162728181484
  (0, 797)	0.025212162728181484
  (0, 872)	0.025212162728181484
  (0, 2201)	0.025212162728181484
  (0, 194)	0.05042432545636297
  (0, 2795)	0.025212162728181484
  (0, 1467)	0.025212162728181484
  (0, 3242)	0.025212162728181484
  :	:
  (252, 1718)	0.12814400000492523
  (252, 1104)	0.11610822413791928
  (252, 457)	0.12814400000492523
  (252, 135)	0.23221644827583857
  (252, 2226)	0.11181246979200793
  (252, 

In [18]:
x_train, x_test, y_train, y_test = train_test_split(XV,Y, test_size= 0.3, random_state=42)

In [26]:
smote = SMOTE(k_neighbors=4)
x_trains,y_trains = smote.fit_resample(x_train,y_train)

print(x_trains.shape)
SMOTEtest = SMOTE(k_neighbors=1)
x_tests,y_tests = SMOTEtest.fit_resample(x_test,y_test)

print(Counter(y_trains))
print(Counter(y_tests))

(590, 3278)
Counter({5: 118, 3: 118, 2: 118, 1: 118, 4: 118})
Counter({1: 45, 5: 45, 4: 45, 2: 45, 3: 45})


In [28]:
print(x_tests.shape)

(225, 3278)


In [20]:
Logistic = LogisticRegression(C=0.01, max_iter=2000, solver='lbfgs')
Logistic.fit(x_trains, y_trains)

LogisticRegression(C=0.01, max_iter=2000)

In [21]:
Category_Predictions = Logistic.predict(x_tests)
logistic_result_category= pd.DataFrame({'Actual': y_tests, 'Predicted': Category_Predictions})
print(logistic_result_category)

     Actual  Predicted
0         1          1
1         5          5
2         5          5
3         1          1
4         4          4
..      ...        ...
220       4          4
221       4          4
222       4          4
223       4          4
224       4          4

[225 rows x 2 columns]


In [22]:
print(classification_report(y_tests, Category_Predictions))

              precision    recall  f1-score   support

           1       0.91      0.89      0.90        45
           2       1.00      0.47      0.64        45
           3       0.81      0.76      0.78        45
           4       0.98      1.00      0.99        45
           5       0.61      0.98      0.75        45

    accuracy                           0.82       225
   macro avg       0.86      0.82      0.81       225
weighted avg       0.86      0.82      0.81       225



In [29]:
vectorizer = TfidfVectorizer(token_pattern=r'\b\w+\b', ngram_range=(1, 2), max_features=5000)
XV = vectorizer.fit_transform(df['tokenized_comments'])
Y= df['Sentiment']

x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(XV,Y, test_size= 0.3, random_state=42)

In [30]:
smote = SMOTE()
x_trains_s,y_trains_s = smote.fit_resample(x_train_s,y_train_s)

print(x_trains_s.shape)
SMOTEtest = SMOTE()
x_tests_s,y_tests_s = SMOTEtest.fit_resample(x_test_s,y_test_s)

print(Counter(y_trains))
print(Counter(y_tests))

(278, 3278)
Counter({5: 118, 3: 118, 2: 118, 1: 118, 4: 118})
Counter({1: 45, 5: 45, 4: 45, 2: 45, 3: 45})


In [31]:
print(x_tests_s.shape)

(124, 3278)


In [25]:
from sklearn.linear_model import LogisticRegressionCV

LogisticS = LogisticRegression(C=10, penalty='l2', solver='liblinear')
LogisticS.fit(x_trains_s, y_trains_s)

LogisticRegression(C=10, solver='liblinear')

In [26]:
Sentiment_Predictions = LogisticS.predict(x_tests_s)
logistic_result_sentiment= pd.DataFrame({'Actual': y_tests_s, 'Predicted': Sentiment_Predictions})
print(logistic_result_sentiment)

     Actual  Predicted
0         1          1
1         1          1
2         1          1
3         1          1
4         1          1
..      ...        ...
119       0          0
120       0          0
121       0          1
122       0          0
123       0          0

[124 rows x 2 columns]


In [27]:
confusion_matrix(y_tests_s, Sentiment_Predictions)

array([[58,  4],
       [ 6, 56]], dtype=int64)

In [28]:
print(classification_report(y_tests_s, Sentiment_Predictions))

              precision    recall  f1-score   support

           0       0.91      0.94      0.92        62
           1       0.93      0.90      0.92        62

    accuracy                           0.92       124
   macro avg       0.92      0.92      0.92       124
weighted avg       0.92      0.92      0.92       124



In [29]:
print("Logistic Regression Category Prediction Accuracy:",
metrics.accuracy_score(y_tests,Category_Predictions)*100)

print("Logistic Regression Sentiment Prediction Accuracy:",
metrics.accuracy_score(y_tests_s,Sentiment_Predictions)*100)

Logistic Regression Category Prediction Accuracy: 81.77777777777779
Logistic Regression Sentiment Prediction Accuracy: 91.93548387096774


In [50]:
vectorizer = TfidfVectorizer(token_pattern=r'\b\w+\b', ngram_range=(1, 2), max_features=5000)
XV = vectorizer.fit_transform(df['tokenized_comments'])
Y= df['Category']

x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(XV,Y, test_size= 0.3, random_state=42)


In [48]:
smote = SMOTE(k_neighbors=4)
x_trains_n,y_trains_n = smote.fit_resample(x_train_n,y_train_n)

SMOTEtest = SMOTE(k_neighbors=1)
x_tests_n,y_tests_n = SMOTEtest.fit_resample(x_test_n,y_test_n)

print(Counter(y_trains))
print(Counter(y_tests))

Counter({5: 118, 3: 118, 2: 118, 1: 118, 4: 118})
Counter({1: 45, 5: 45, 4: 45, 2: 45, 3: 45})


In [32]:
svmC = LinearSVC(C=0.01)
svmC.fit(x_trains_n, y_trains_n)

LinearSVC(C=0.01)

In [33]:
Category_Predictions_SVC = svmC.predict(x_tests_n)
SVC_result_category= pd.DataFrame({'Actual': y_tests_n, 'Predicted': Category_Predictions_SVC})
print(SVC_result_category)

     Actual  Predicted
0         1          1
1         5          5
2         5          5
3         1          1
4         4          4
..      ...        ...
220       4          4
221       4          4
222       4          4
223       4          4
224       4          4

[225 rows x 2 columns]


In [34]:
print(classification_report(y_tests_n,Category_Predictions_SVC))

              precision    recall  f1-score   support

           1       0.88      0.96      0.91        45
           2       1.00      0.60      0.75        45
           3       0.88      0.84      0.86        45
           4       0.98      1.00      0.99        45
           5       0.72      0.96      0.82        45

    accuracy                           0.87       225
   macro avg       0.89      0.87      0.87       225
weighted avg       0.89      0.87      0.87       225



In [35]:
vectorizer = TfidfVectorizer(token_pattern=r'\b\w+\b', ngram_range=(1, 2), max_features=5000)
XV = vectorizer.fit_transform(df['tokenized_comments'])
Y= df['Sentiment']


x_train_ns, x_test_ns, y_train_ns, y_test_ns = train_test_split(XV,Y, test_size= 0.3, random_state=42)

In [36]:
smote = SMOTE()
x_trains_ns,y_trains_ns = smote.fit_resample(x_train_ns,y_train_ns)

SMOTEtest = SMOTE()
x_tests_ns,y_tests_ns = SMOTEtest.fit_resample(x_test_ns,y_test_ns)

In [37]:
svmS = LinearSVC(C=0.1)

svmS.fit(x_trains_ns, y_trains_ns)

LinearSVC(C=0.1)

In [38]:
Sentiment_Predictions_SVC = svmS.predict(x_tests_ns)
SVC_result_sentiment= pd.DataFrame({'Actual': y_tests_ns, 'Predicted': Sentiment_Predictions_SVC})
print(SVC_result_sentiment)

     Actual  Predicted
0         1          1
1         1          1
2         1          1
3         1          1
4         1          1
..      ...        ...
119       0          1
120       0          1
121       0          0
122       0          0
123       0          0

[124 rows x 2 columns]


In [39]:
print(classification_report(y_tests_ns,Sentiment_Predictions_SVC))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93        62
           1       0.89      1.00      0.94        62

    accuracy                           0.94       124
   macro avg       0.94      0.94      0.94       124
weighted avg       0.94      0.94      0.94       124



In [40]:
print("Logistic Regression Category Prediction Accuracy:",
metrics.accuracy_score(y_tests,Category_Predictions)*100)
print("Logistic Regression Sentiment Prediction Accuracy:",
metrics.accuracy_score(y_tests_s,Sentiment_Predictions)*100)

print("SVC Category Prediction Accuracy:",
metrics.accuracy_score(y_tests_n,Category_Predictions_SVC)*100)
print("SVC Sentiment Prediction Accuracy:",
metrics.accuracy_score(y_tests_ns,Sentiment_Predictions_SVC)*100)

Logistic Regression Category Prediction Accuracy: 81.77777777777779
Logistic Regression Sentiment Prediction Accuracy: 91.93548387096774
SVC Category Prediction Accuracy: 87.1111111111111
SVC Sentiment Prediction Accuracy: 93.54838709677419
